In [11]:
import json
import pandas as pd

def process_tags(tags): 
    tags = list(map(int, tags.split(",")))
    return tags

bound = 50
head_pred_path = "../_EXPERIMENTS/head50__transformer__attn_pooling__flip_cut_aug__lion__weighted_bce_1/prediction.csv"
tail_pred_path = "../_EXPERIMENTS/tail50__transformer__attn_pooling__flip_cut_aug__lion__weighted_bce_1/prediction.csv"

head = pd.read_csv(head_pred_path)
tail = pd.read_csv(tail_pred_path)

with open(f"../data/head{bound}_label2tag.json") as f:
    head_id2tag = json.load(f)

with open(f"../data/tail{bound}_label2tag.json") as f:
    tail_id2tag = json.load(f)


predictions = [{} for _ in range(len(head))]

for i, p in enumerate(head.prediction.values):
    raw_preds = list(map(float, p.split(",")))
    preds = {head_id2tag[str(j)]: proba for j, proba in enumerate(raw_preds)}
    predictions[i].update(preds)

for i, p in enumerate(tail.prediction.values):
    raw_preds = list(map(float, p.split(",")))
    preds = {tail_id2tag[str(j)]: proba for j, proba in enumerate(raw_preds)}
    predictions[i].update(preds)

def process_prediction(pred):
    res = [""] * 256
    for i, p in pred.items():
        res[i] = str(p)
    return ",".join(res)

result = pd.DataFrame(
    {
        "track": head.track,
        "prediction": [process_prediction(p) for p in predictions]
    }
)
result.to_csv(f"head_tail_prediction_bound{bound}.csv", index=False)